<a href="https://colab.research.google.com/github/zerxe/2D_Game_PGL/blob/main/SNS/SNS_Actividad_3_3_%E2%80%93_IRIS_con_NaiveBayes_y_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adolfo Morales Colmenares

## Imports

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB, CategoricalNB
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer
from sklearn.metrics import accuracy_score
import numpy as np

## IRIS

In [6]:
# Cargar datos Iris
iris = load_iris()
X, y = iris.data, iris.target

# Preprocesamiento
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
discretizer = KBinsDiscretizer(n_bins=9, encode='ordinal', strategy='uniform')
X_discrete = discretizer.fit_transform(X_scaled)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inicializar clasificadores
models = {
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "CategoricalNB": CategoricalNB()
}

# Evaluación de los modelos
results = {}
for name, model in models.items():
    try:
        # Sin validación cruzada
        if name in ["MultinomialNB", "BernoulliNB", "ComplementNB", "CategoricalNB"]:
            model.fit(X_discrete, y)
            accuracy_no_cv = accuracy_score(y_test, model.predict(discretizer.transform(X_test)))
        else:
            model.fit(X_train, y_train)
            accuracy_no_cv = accuracy_score(y_test, model.predict(X_test))

        # Con validación cruzada
        if name in ["MultinomialNB", "BernoulliNB", "ComplementNB", "CategoricalNB"]:
            cv_scores = cross_val_score(model, X_discrete, y, cv=5)
        else:
            cv_scores = cross_val_score(model, X, y, cv=5)

        results[name] = {
            "Sin CrossValidation": round(accuracy_no_cv, 2),
            "Con CrossValidation": round(np.mean(cv_scores), 2)
        }
    except Exception as e:
        results[name] = {
            "Sin CrossValidation": "Error",
            "Con CrossValidation": "Error"
        }
        print(f"Error con {name}: {e}")

## Resultado

In [7]:
# Mostrar resultados en formato tabla
print("\nResultados para Iris:")
print(f"{'Modelo':<15} {'Sin CrossValidation':<20} {'Con CrossValidation':<20}")
for model, scores in results.items():
    print(f"{model:<15} {scores['Sin CrossValidation']:<20} {scores['Con CrossValidation']:<20}")


Resultados para Iris:
Modelo          Sin CrossValidation  Con CrossValidation 
GaussianNB      0.98                 0.95                
MultinomialNB   0.29                 0.75                
BernoulliNB     0.36                 0.65                
ComplementNB    0.42                 0.67                
CategoricalNB   0.36                 0.95                
